# calculate ACC and NMAE within each LME along the vertical direction

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4
import numpy as np
from scipy.stats import pearsonr
from matplotlib import cm,colors,gridspec
import scipy.signal as sgl
import cmcrameri.cm as cmc
import os

In [2]:
USER = os.environ['USER']
var = 'Phi'
tres = 'yearly'

## FOSI hindcast 

In [3]:
datadir = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{datadir}/DPLE-FOSI_hindcast_{var}_Eo_space_horizontal_mean_LME_1948-2017_{tres}.nc'
ds_hind = xr.open_dataset(filename)
ds_hind

<xarray.Dataset>
Dimensions:  (lme: 11, time: 70, z_t: 35)
Coordinates:
  * lme      (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 ... 60 61 62 63 64 65 66 67 68 69
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
Data variables: (12/13)
    Phi_0    (lme, time, z_t) float64 ...
    Phi_1    (lme, time, z_t) float64 ...
    Phi_2    (lme, time, z_t) float64 ...
    Phi_3    (lme, time, z_t) float64 ...
    Phi_4    (lme, time, z_t) float64 ...
    Phi_5    (lme, time, z_t) float64 ...
    ...       ...
    Phi_7    (lme, time, z_t) float64 ...
    Phi_8    (lme, time, z_t) float64 ...
    Phi_9    (lme, time, z_t) float64 ...
    Phi_10   (lme, time, z_t) float64 ...
    Phi_11   (lme, time, z_t) float64 ...
    Phi_12   (lme, time, z_t) float64 ...

## DPLE forecast

In [4]:
filename = f'{datadir}/DPLE_driftcorrected_{var}_Eo_space_horizontal_mean_LME_ens_mean_yearly.nc'
ds_fore = xr.open_dataset(filename)
ds_fore

<xarray.Dataset>
Dimensions:  (lme: 11, Y: 64, year: 10, z_t: 35)
Coordinates:
  * lme      (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y        (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * year     (year) int64 0 1 2 3 4 5 6 7 8 9
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
Data variables: (12/13)
    Phi_0    (lme, Y, year, z_t) float64 ...
    Phi_1    (lme, Y, year, z_t) float64 ...
    Phi_2    (lme, Y, year, z_t) float64 ...
    Phi_3    (lme, Y, year, z_t) float64 ...
    Phi_4    (lme, Y, year, z_t) float64 ...
    Phi_5    (lme, Y, year, z_t) float64 ...
    ...       ...
    Phi_7    (lme, Y, year, z_t) float64 ...
    Phi_8    (lme, Y, year, z_t) float64 ...
    Phi_9    (lme, Y, year, z_t) float64 ...
    Phi_10   (lme, Y, year, z_t) float64 ...
    Phi_11   (lme, Y, year, z_t) float64 ...
    Phi_12   (lme, Y, year, z_t) float64 ...

## Calculate Corr/Rcri Matrix for Lead Year v. LME box#

In [5]:
import tools
alpha = 0.05
# time period for evaluation 1954-2007
yr1 = 1954
yr2 = 2007
# lead year range: 1,2,...,9,10
ldyrs = range(1, 11)
#
nver = range(35)
# persistence ACC: var1; var2
# DPLE ACC:        var2; var3
lme = [1,2,3,4,5,6,7,8,9,10,65]

lmen = ['EBS','GoA','CC','GoC','GoM','SEUS','NEUS','SS','LN','IPH','AI']
# 9/13 phis
nphi = 13
#
corr_p = np.ma.zeros([nphi, len(lme), len(ldyrs), len(nver)]); corr_p.mask=True
rcri_p = np.ma.zeros([nphi, len(lme), len(ldyrs), len(nver)]); rcri_p.mask=True
corr_d = corr_p.copy()
rcri_d = rcri_p.copy()
corr13 = corr_p.copy()
rcri13 = rcri_p.copy()
nmae_p = corr_p.copy()
nmae_d = corr_p.copy()
#
for cc in range(13):
    vv = cc
    for nn in range(len(lme)):
        box = lme[nn]
        var1 = ds_hind[f'{var}_{vv}'].sel(lme=box)[yr1-1948:yr2-1948+1].values
        var1c = np.tile(np.nanmean(var1,axis=0)[np.newaxis,:],[yr2-yr1+1,1])
        var1 = var1 - var1c
        #
        for lead in range(1, 11):
            var2 = ds_hind[f'{var}_{vv}'].sel(lme=box)[yr1-1948+lead:yr2-1948+1+lead].values
            var2c = np.tile(np.nanmean(var2,axis=0)[np.newaxis,:],[yr2-yr1+1,1])
            var2 = var2 - var2c
            var3 = ds_fore[f'{var}_{cc}'].sel(lme=box)[yr1-1954:yr2-1954+1, lead-1].values
            var3c = np.tile(np.nanmean(var3,axis=0)[np.newaxis,:],[yr2-yr1+1,1])
            var3 = var3 - var3c
            #
            for zz in nver:
                # mask nan values
                mask1 = np.isnan(var1[:,zz])
                mask2 = np.isnan(var2[:,zz])
                mask3 = np.isnan(var3[:,zz])
                mem1 = np.ma.array(var1[:,zz], mask=mask1)
                mem2 = np.ma.array(var2[:,zz], mask=mask2)
                mem3 = np.ma.array(var3[:,zz], mask=mask3)
                # Persistence ACC & NMAE
                fmask = np.logical_and(mem1.mask, mem2.mask)
                if np.sum(fmask) != len(fmask):
                    nen1 = mem1[~fmask]
                    nen2 = mem2[~fmask]
                    corr_p[cc, nn, lead-1, zz] = pearsonr(nen1, nen2)[0]
                    neff = tools.neff3(nen1, nen2, len(nen1))
                    rcri_p[cc, nn, lead-1, zz] = tools.calculate_parson_corr_critical_value(neff, alpha)
                    #
                    nmae_p[cc, nn, lead-1, zz] = np.nanmean(np.abs(nen1-nen2)/np.nanstd(nen2))
                # DPLE ACC & NMAE
                fmask = np.logical_and(mem2.mask, mem3.mask)
                if np.sum(fmask) != len(fmask):
                    nen2 = mem2[~fmask]
                    nen3 = mem3[~fmask]
                    corr_d[cc, nn, lead-1, zz] = pearsonr(nen2, nen3)[0]
                    neff = tools.neff3(nen2, nen3, len(nen2))
                    rcri_d[cc, nn, lead-1, zz] = tools.calculate_parson_corr_critical_value(neff, alpha)
                    #
                    nmae_d[cc, nn, lead-1, zz] = np.nanmean(np.abs(nen3-nen2)/np.nanstd(nen2))
                # Correlation for ACC Difference Significance Test
                fmask = np.logical_and(mem1.mask, mem3.mask)
                if np.sum(fmask) != len(fmask):
                    nen1 = mem1[~fmask]
                    nen3 = mem3[~fmask]
                    corr13[cc, nn, lead-1, zz] = pearsonr(nen1, nen3)[0]
                    neff = tools.neff3(nen1, nen3, len(nen1))
                    rcri13[cc, nn, lead-1, zz] = tools.calculate_parson_corr_critical_value(neff, alpha)
        #

In [6]:
data = xr.Dataset({"corr_p": (("nphi","nbox","nlead","nver"),corr_p),\
                   "rcri_p": (("nphi","nbox","nlead","nver"),rcri_p),\
                   "corr_d": (("nphi","nbox","nlead","nver"),corr_d),\
                   "rcri_d": (("nphi","nbox","nlead","nver"),rcri_d),\
                   "corr13": (("nphi","nbox","nlead","nver"),corr13),\
                   "rcri13": (("nphi","nbox","nlead","nver"),rcri13),\
                   "nmae_p": (("nphi","nbox","nlead","nver"),nmae_p),\
                   "nmae_d": (("nphi","nbox","nlead","nver"),nmae_d)})

In [7]:
dout = f'/glade/scratch/{USER}/DPLE-results'
data.load()

<xarray.Dataset>
Dimensions:  (nphi: 13, nbox: 11, nlead: 10, nver: 35)
Dimensions without coordinates: nphi, nbox, nlead, nver
Data variables:
    corr_p   (nphi, nbox, nlead, nver) float64 0.5042 0.5121 ... -0.1035 -0.12
    rcri_p   (nphi, nbox, nlead, nver) float64 0.3494 0.355 0.355 ... 0.95 0.95
    corr_d   (nphi, nbox, nlead, nver) float64 0.5024 0.5016 ... 0.5017 0.4982
    rcri_d   (nphi, nbox, nlead, nver) float64 0.344 0.344 ... 0.7545 0.8114
    corr13   (nphi, nbox, nlead, nver) float64 0.806 0.8105 ... -0.02519
    rcri13   (nphi, nbox, nlead, nver) float64 0.344 0.344 ... 0.7545 0.8114
    nmae_p   (nphi, nbox, nlead, nver) float64 0.8171 0.8061 ... 1.274 1.287
    nmae_d   (nphi, nbox, nlead, nver) float64 0.7106 0.7072 ... 0.6559 0.6643

In [8]:
data.to_netcdf(f'{dout}/DPLE_LME_vertical_box01-11_ACC_NMAE_{var}_Eo_space_leadyear_1-10.nc', mode='w')

### save source data for Figure 4 

In [9]:
data = xr.Dataset({"corr_p": (("nbox","nlead","nver"),corr_p[6]),\
                   "rcri_p": (("nbox","nlead","nver"),rcri_p[6]),\
                   "corr_d": (("nbox","nlead","nver"),corr_d[6]),\
                   "rcri_d": (("nbox","nlead","nver"),rcri_d[6]),\
                   "corr13": (("nbox","nlead","nver"),corr13[6]),\
                   "rcri13": (("nbox","nlead","nver"),rcri13[6]),\
                   "nmae_p": (("nbox","nlead","nver"),nmae_p[6]),\
                   "nmae_d": (("nbox","nlead","nver"),nmae_d[6])})

In [10]:
data.load()
data.to_netcdf(f'{dout}/DPLE_LME_vertical_box01-11_ACC_NMAE_{var}_medium_Eo_leadyear_1-10.nc', mode='w')